## Generate urban morphological indices automatically with *Morphoindex_generator*

This section showcases some very easy-to-use methods to generate morphological indices automatically from the built environment Shapefiles with our `morphoindex_generator`.

First, as an example, let us have a look at the original grids Shapefile in Singapore.

In [1]:
import geopandas as gpd

In [2]:
grid_path = './data/Singapore/grids/grids.shp'
grids = gpd.read_file(grid_path)

In [3]:
grids

,id,geometry
0,Singapore_0,"POLYGON ((103.81542 1.39125, 103.81542 1.39958..."
1,Singapore_1,"POLYGON ((103.82375 1.39125, 103.82375 1.39958..."
2,Singapore_2,"POLYGON ((103.83208 1.39125, 103.83208 1.39958..."
3,Singapore_3,"POLYGON ((103.84042 1.39125, 103.84042 1.39958..."
4,Singapore_4,"POLYGON ((103.84875 1.39125, 103.84875 1.39958..."
...,...,...
838,Singapore_838,"POLYGON ((103.84042 1.40792, 103.84042 1.41625..."
839,Singapore_839,"POLYGON ((103.84875 1.40792, 103.84875 1.41625..."
840,Singapore_840,"POLYGON ((103.81542 1.39958, 103.81542 1.40792..."
841,Singapore_841,"POLYGON ((103.82375 1.39958, 103.82375 1.40792..."


### Traditional morphological indices
The grid Shapefile for now seems to only contains the information of grid IDs and geometries. No worries! In the next step, we would show a very simple approach to generate the most popular traditioanl urban morphological indices with our `morphoindex_generator`.

In [4]:
from morphoindex_generator import get_MorphoIndex

In [5]:
# Specify the filepaths of the Shapefiles.

road_path = './data/Singapore/osm_data.shp/gis_osm_roads_free_1.shp'
building_path = './data/Singapore/osm_data.shp/gis_osm_buildings_a_free_1.shp'
landuse_path = './data/Singapore/osm_data.shp/gis_osm_landuse_a_free_1.shp'

# If you want to save the results as a new file, you should specifiy the save_path, or the default would be saving inplace

save_path =  './data/Singapore/grids/grids_morpho.shp'

# Get the morphological indices with just a single line of code.

grids = get_MorphoIndex(grid_path, road_path, building_path, landuse_path, save_path, 
                       get_intersection=False,  # whether to calculate intersection density
                       drop_nonbuilt=True  # whether to drop grids without buildings and roads.
                      )

Start getting mophoindex
--------------------------------------------------
Grids loaded!
Road completed!
Building completed!
Block completed!
--------------------------------------------------
Dropping non-built grids
--------------------------------------------------


In [6]:
grids

,geometry,RD,BuD,ABFA,BID,ABA,LUM
id,,,,,,,
Singapore_0,"POLYGON ((103.81542 1.39125, 103.81542 1.39958...",15095.294027,68798.049109,312.718405,452,1904.453211,1.167136
Singapore_1,"POLYGON ((103.82375 1.39125, 103.82375 1.39958...",7978.665299,16632.388881,264.006173,118,7295.024130,0.793388
Singapore_2,"POLYGON ((103.83208 1.39125, 103.83208 1.39958...",14645.738839,37367.120902,983.345287,72,11955.733994,1.760057
Singapore_3,"POLYGON ((103.84042 1.39125, 103.84042 1.39958...",10408.896512,10955.844936,2738.961234,15,57387.523153,0.631043
Singapore_4,"POLYGON ((103.84875 1.39125, 103.84875 1.39958...",16587.969485,19022.800557,442.390711,89,9672.054469,0.899144
...,...,...,...,...,...,...,...
Singapore_838,"POLYGON ((103.84042 1.40792, 103.84042 1.41625...",15232.272905,64759.821093,1044.513243,16,53801.186460,1.041339
Singapore_839,"POLYGON ((103.84875 1.40792, 103.84875 1.41625...",4212.374482,44.569254,44.569254,9,95646.553707,0.914891
Singapore_840,"POLYGON ((103.81542 1.39958, 103.81542 1.40792...",16284.978937,143651.396390,353.821173,705,1221.015450,1.833610


Now you get the morphological indices calculated for each grid. All you need to do are to provide the filepaths of your road, building and land use Shapefiles, and then wait for the outcome with patience. The indices include:

* RD -> Road density
* Bud -> Building density
* ABFA -> Average building footprint area
* BlD -> Block density
* ABA -> Average building area
* LUM -> Land use mixture
* ID -> Intersection density

You can also get another index ID (Intersection density) if you set the parameter `get_intersection` to `True`. It should be noted that the calculation is based on `OSMnx`. It would cost a few time to do so.

### Categorical probabilities

Besides traditional morphological indices, `morphoindex generator` also provides east-to-use method to get road network categorical probabilities.

In [2]:
from morphoindex_generator import prob_calculator

In [3]:
# # Specify the filepaths.
model_path = './model/ResNet34-4class6/ResNet34-4class6.h5'  # filepath of your model
grid_path = './data/Singapore/grids/grids.shp'
image_path = './data/Singapore/images/'   # filepath of your CRHD images
save_path = './data/Singapore/grids/grids_probs.shp'

pc = prob_calculator(channeles=3)  # Define a prob_calculator that deal with images of three channeles
pc.load_model(model_path)  # Load the road network classification model
grids = pc.getProb(grid_path, image_path, save_path)

Current grid: Singapore_0
Current grid: Singapore_100
Current grid: Singapore_200
Current grid: Singapore_300
Current grid: Singapore_400
Current grid: Singapore_500
Current grid: Singapore_600
trying to plot grid Singapore_657
Current grid: Singapore_700
trying to plot grid Singapore_778
Current grid: Singapore_800
Completed!


In [6]:
grids.head(10)

,geometry,Prob_O,Prob_N,Prob_G,Prob_R,cls
id,,,,,,
Singapore_0,"POLYGON ((103.81542 1.39125, 103.81542 1.39958...",0.021126,0.969007,0.000731,0.009135,Nopattern
Singapore_1,"POLYGON ((103.82375 1.39125, 103.82375 1.39958...",0.000129,0.999788,0.000009,0.000074,Nopattern
Singapore_2,"POLYGON ((103.83208 1.39125, 103.83208 1.39958...",0.006211,0.988107,0.000693,0.004989,Nopattern
Singapore_3,"POLYGON ((103.84042 1.39125, 103.84042 1.39958...",0.006552,0.989585,0.000789,0.003074,Nopattern
Singapore_4,"POLYGON ((103.84875 1.39125, 103.84875 1.39958...",0.133779,0.769552,0.079368,0.017302,Nopattern
Singapore_5,"POLYGON ((103.85708 1.39125, 103.85708 1.39958...",0.811697,0.021005,0.130300,0.036997,Organic
Singapore_6,"POLYGON ((103.81542 1.38292, 103.81542 1.39125...",0.013242,0.979593,0.000250,0.006915,Nopattern
Singapore_7,"POLYGON ((103.82375 1.38292, 103.82375 1.39125...",0.022905,0.963501,0.000579,0.013015,Nopattern
Singapore_8,"POLYGON ((103.83208 1.38292, 103.83208 1.39125...",0.702169,0.145911,0.016265,0.135656,Organic


We have optained the probabilities and the classes now! They are ready to be dumped into other models as inputs. The method would generate the CRHD image right away if there is not a matched CRHD image for a grid。